In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [30]:
import pandas as pd


da_2018 = pd.read_csv(r'C:\Users\DELL\Desktop\NATPAC\Prediction\Data\c_data_18.csv')

da_2019 = pd.read_csv(r'C:\Users\DELL\Desktop\NATPAC\Prediction\Data\c_data_19.csv')

da_2020 = pd.read_csv(r'C:\Users\DELL\Desktop\NATPAC\Prediction\Data\c_data_20.csv')

da_2021 = pd.read_csv(r'C:\Users\DELL\Desktop\NATPAC\Prediction\Data\c_data_21.csv')



In [31]:
da_2018 = da_2018.drop_duplicates()
da_2019 = da_2019.drop_duplicates()
da_2020 = da_2020.drop_duplicates()
da_2021 = da_2021.drop_duplicates()


In [32]:
da_2018.shape

(7179, 2)

In [33]:
da_2019.shape

(8809, 2)

In [34]:
da_2020.shape

(5962, 2)

In [35]:
da_2021.shape

(7094, 2)

In [36]:
merged_data= pd.concat([da_2018,da_2019,da_2020,da_2021])
merged_data


,Sail Date.,QTY.(KGS)
0,2018-01-01,13135
1,2018-01-01,15876
2,2018-01-01,71366
3,2018-01-01,14000
4,2018-01-01,24000
...,...,...
8100,2021-10-31,26000
8102,2021-10-31,57600
8103,2021-10-31,50096
8104,2021-10-31,49160


In [37]:
merged_data.size

58088

In [38]:
da_2018.size

14358

In [39]:
da_2019.size

17618

In [40]:
da_2020.size

11924

In [41]:
da_2021.size

14188

In [42]:
# Check for multiple entries for the same date and aggregate the values
merged_data_aggregated = merged_data.groupby(merged_data.index).sum()

# Reset the index to make the date column a regular column
merged_data_aggregated.reset_index(inplace=True)

# Display the aggregated data
print(merged_data_aggregated)


       index                                Sail Date.  QTY.(KGS)
0          0  2018-01-012019-01-012020-01-012021-01-01      90631
1          1  2018-01-012019-01-012020-01-012021-01-01     106436
2          2  2018-01-012019-01-012020-01-012021-01-01     125133
3          3  2018-01-012019-01-012020-01-012021-01-01      99153
4          4            2018-01-012019-01-012021-01-01      87249
...      ...                                       ...        ...
10903  11716                                2019-12-31      35363
10904  11717                                2019-12-31      27516
10905  11718                                2019-12-31       2140
10906  11720                                2019-12-31      17433
10907  11721                                2019-12-31      14560

[10908 rows x 3 columns]


In [43]:
merged_data.size

merged_data.to_csv('combined_set.csv', index=False)



In [44]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Read the combined dataset from the CSV file
merged_data = pd.read_csv('combined_set.csv')

# Convert 'Sail Date.' to datetime format
merged_data['Sail Date.'] = pd.to_datetime(merged_data['Sail Date.'])

# Extract year, month, and day as numerical features
merged_data['Year'] = merged_data['Sail Date.'].dt.year
merged_data['Month'] = merged_data['Sail Date.'].dt.month
merged_data['Day'] = merged_data['Sail Date.'].dt.day

# Drop the original 'Sail Date.' column
merged_data.drop(columns=['Sail Date.'], inplace=True)

# Normalize the features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merged_data)

# Convert the DataFrame to a numpy array
data_array = np.array(scaled_data)

# Define the sequence length (number of time steps)
sequence_length = 3  # Assuming you want to use past 3 months' data to predict the next month

# Create sequences for input and target variables
X = []
y = []
for i in range(len(data_array) - sequence_length):
    X.append(data_array[i:i+sequence_length])
    y.append(data_array[i+sequence_length, -1])  # Assuming the last column is the target variable

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

# Define RNN model
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=50, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    tf.keras.layers.Dense(units=1)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=100, verbose=1)

# Evaluate model
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error on test data:", mse)




Epoch 1/100

726/726 [==============================] - 3s 2ms/step - loss: 0.0019
Epoch 2/100
726/726 [==============================] - 1s 2ms/step - loss: 4.6685e-04
Epoch 3/100
726/726 [==============================] - 1s 2ms/step - loss: 4.2587e-04
Epoch 4/100
726/726 [==============================] - 1s 2ms/step - loss: 4.2624e-04
Epoch 5/100
726/726 [==============================] - 1s 2ms/step - loss: 4.0891e-04
Epoch 6/100
726/726 [==============================] - 1s 2ms/step - loss: 4.0734e-04
Epoch 7/100
726/726 [==============================] - 1s 2ms/step - loss: 4.1011e-04
Epoch 8/100
726/726 [==============================] - 1s 2ms/step - loss: 4.0431e-04
Epoch 9/100
726/726 [==============================] - 1s 2ms/step - loss: 4.0181e-04
Epoch 10/100
726/726 [==============================] - 1s 2ms/step - loss: 3.9524e-04
Epoch 11/100
726/726 [==============================] - 1s 2ms/step - loss: 4.0172e-04
Epoch 12/100
726/726 [==============================]

In [48]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Read the combined dataset from the CSV file
merged_data = pd.read_csv('combined_set.csv')

# Convert 'Sail Date.' to datetime format
merged_data['Sail Date.'] = pd.to_datetime(merged_data['Sail Date.'])

# Extract year, month, and day as numerical features
merged_data['Year'] = merged_data['Sail Date.'].dt.year
merged_data['Month'] = merged_data['Sail Date.'].dt.month
merged_data['Day'] = merged_data['Sail Date.'].dt.day

# Drop the original 'Sail Date.' column
merged_data.drop(columns=['Sail Date.'], inplace=True)

# Normalize the features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merged_data)

# Define the sequence length (number of time steps)
sequence_length = 3  # Assuming you want to use past 3 months' data to predict the next month

# Create sequences for input and target variables
X = []
y = []
for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length])
    y.append(scaled_data[i+sequence_length, -1])  # Assuming the last column is the target variable

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define RNN model
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=50, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    tf.keras.layers.Dense(units=1)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=100, verbose=1)

# Evaluate model
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error on test data:", mse)

# Make predictions
y_pred = model.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


Epoch 1/100
726/726 [==============================] - 2s 2ms/step - loss: 0.0065
Epoch 2/100
726/726 [==============================] - 1s 2ms/step - loss: 4.9243e-04
Epoch 3/100
726/726 [==============================] - 1s 2ms/step - loss: 4.3431e-04
Epoch 4/100
726/726 [==============================] - 1s 2ms/step - loss: 4.1101e-04
Epoch 5/100
726/726 [==============================] - 1s 2ms/step - loss: 4.2423e-04
Epoch 6/100
726/726 [==============================] - 2s 2ms/step - loss: 4.0028e-04
Epoch 7/100
726/726 [==============================] - 2s 2ms/step - loss: 4.0156e-04
Epoch 8/100
726/726 [==============================] - 1s 2ms/step - loss: 3.7799e-04
Epoch 9/100
726/726 [==============================] - 1s 2ms/step - loss: 3.8074e-04
Epoch 10/100
726/726 [==============================] - 2s 2ms/step - loss: 3.8429e-04
Epoch 11/100
726/726 [==============================] - 1s 2ms/step - loss: 3.7706e-04
Epoch 12/100
726/726 [==============================] - 

In [65]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Read the combined dataset from the CSV file
merged_data = pd.read_csv('combined_set.csv')

# Convert 'Sail Date.' to datetime format
merged_data['Sail Date.'] = pd.to_datetime(merged_data['Sail Date.'])

# Extract year, month, and day as numerical features
merged_data['Year'] = merged_data['Sail Date.'].dt.year
merged_data['Month'] = merged_data['Sail Date.'].dt.month
merged_data['Day'] = merged_data['Sail Date.'].dt.day

# Drop the original 'Sail Date.' column
merged_data.drop(columns=['Sail Date.'], inplace=True)

# Normalize the features
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(merged_data)

# Define the sequence length (number of time steps)
sequence_length = 3  # Assuming you want to use past 3 months' data to predict the next month

# Create sequences for input and target variables
X = []
y = []
for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length])
    y.append(scaled_data[i+sequence_length, -1])  # Assuming the last column is the target variable

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define RNN model
model = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(units=50, activation='relu', input_shape=(X.shape[1], X.shape[2])),
    tf.keras.layers.Dense(units=1)
])

# Compile model
model.compile(optimizer='adam', loss='mse')

# Train model
model.fit(X_train, y_train, epochs=100, verbose=1)

# Evaluate model
mse = model.evaluate(X_test, y_test)
print("Mean Squared Error on test data:", mse)

Epoch 1/100
726/726 [==============================] - 4s 4ms/step - loss: 0.0024
Epoch 2/100
726/726 [==============================] - 3s 4ms/step - loss: 4.5713e-04
Epoch 3/100
726/726 [==============================] - 3s 4ms/step - loss: 4.1796e-04
Epoch 4/100
726/726 [==============================] - 3s 4ms/step - loss: 3.9840e-04
Epoch 5/100
726/726 [==============================] - 2s 3ms/step - loss: 3.9000e-04
Epoch 6/100
726/726 [==============================] - 3s 4ms/step - loss: 3.9049e-04
Epoch 7/100
726/726 [==============================] - 2s 2ms/step - loss: 3.9122e-04
Epoch 8/100
726/726 [==============================] - 3s 4ms/step - loss: 3.7233e-04
Epoch 9/100
726/726 [==============================] - 3s 4ms/step - loss: 3.6370e-04
Epoch 10/100
726/726 [==============================] - 3s 4ms/step - loss: 3.6381e-04
Epoch 11/100
726/726 [==============================] - 3s 4ms/step - loss: 3.7056e-04
Epoch 12/100
726/726 [==============================] - 

In [64]:
# Define the start and end dates for 2022
start_date_2022 = pd.Timestamp(2022, 1, 1)
end_date_2022 = pd.Timestamp(2022, 12, 31)

# Generate a date range for 2022
date_range_2022 = pd.date_range(start=start_date_2022, end=end_date_2022)

# Initialize an empty array to store predicted sales quantities for 2022
predicted_sales_2022 = []

# Initialize the last sequence with the last known data from the test set
last_sequence = X_test[-1:]

# Iterate over each date in 2022
for i, date in enumerate(date_range_2022):
    # Predict the sales quantity for the next date
    predicted_sales_scaled = model.predict(last_sequence)
    
    # Perform inverse transformation to get the predicted sales quantity in original scale
    predicted_sales = scaler.inverse_transform(np.array([[predicted_sales_scaled[0][0], 0, 0, 0]]))
    
    # Store the predicted sales quantity for the current date
    predicted_sales_2022.append(predicted_sales[0][0])
    
    # Update the last sequence with the predicted sales quantity for the next date
    last_sequence = np.roll(last_sequence, -1, axis=1)  # Shift elements to the left by one
    last_sequence[0, -1, 0] = predicted_sales_scaled[0][0]  # Update the last element of the last sequence

# Create a DataFrame to store the predicted sales quantities with dates as index
predicted_sales_df = pd.DataFrame(predicted_sales_2022, index=date_range_2022, columns=['Predicted Sales Quantity'])

print("Predicted sales quantities for 2022:")
print(predicted_sales_df)


1/1 [==============================] - 0s 50ms/step
Predicted sales quantities for 2022:
            Predicted Sales Quantity
2022-01-01              82325.494251
2022-01-02              62879.972373
2022-01-03              85931.475355
2022-01-04              82921.202991
2022-01-05              62561.787840
...                              ...
2022-12-27              82906.033344
2022-12-28              62564.216874
2022-12-29              86346.102929
2022-12-30              82906.033344
2022-12-31              62564.216874

[365 rows x 1 columns]
